In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf

import warnings
from math import sqrt
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.tseries.offsets import DateOffset

In [2]:
fd = pd.read_csv("Chennai_1990_2022_Madras.csv")
fd

,time,tavg,tmin,tmax,prcp
0,01-01-1990,25.2,22.8,28.4,0.5
1,02-01-1990,24.9,21.7,29.1,0.0
2,03-01-1990,25.6,21.4,29.8,0.0
3,04-01-1990,25.7,NaN,28.7,0.0
4,05-01-1990,25.5,20.7,28.4,0.0
...,...,...,...,...,...
11889,21-07-2022,28.4,24.5,32.8,21.1
11890,22-07-2022,27.8,24.6,32.2,22.1
11891,23-07-2022,27.4,24.7,32.6,18.6
11892,24-07-2022,27.8,25.0,33.3,9.1


In [3]:
fd.isnull().sum()

time       0
tavg      27
tmin    3084
tmax    1019
prcp    4886
dtype: int64

In [4]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11894 entries, 0 to 11893
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    11894 non-null  object 
 1   tavg    11867 non-null  float64
 2   tmin    8810 non-null   float64
 3   tmax    10875 non-null  float64
 4   prcp    7008 non-null   float64
dtypes: float64(4), object(1)
memory usage: 464.7+ KB


In [5]:
fd.head()

,time,tavg,tmin,tmax,prcp
0,01-01-1990,25.2,22.8,28.4,0.5
1,02-01-1990,24.9,21.7,29.1,0.0
2,03-01-1990,25.6,21.4,29.8,0.0
3,04-01-1990,25.7,NaN,28.7,0.0
4,05-01-1990,25.5,20.7,28.4,0.0


In [6]:
dateparse = lambda x : pd.to_datetime(x, format='%d-%m-%Y',errors='coerce')
df = pd.read_csv('Chennai_1990_2022_Madras.csv',parse_dates = ['time'],index_col='time',date_parser=dateparse)

/tmp/ipykernel_6255/2987536676.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('Chennai_1990_2022_Madras.csv',parse_dates = ['time'],index_col='time',date_parser=dateparse)


In [7]:
df= df.drop(['tmin','tmax','prcp'],axis=1)

In [8]:
df.head()

,tavg
time,
1990-01-01,25.2
1990-01-02,24.9
1990-01-03,25.6
1990-01-04,25.7
1990-01-05,25.5


In [9]:
df.isna().sum()

tavg    27
dtype: int64

In [10]:
df['tavg'] = df['tavg'].interpolate(method='linear')

In [11]:
df['tavg'] = df['tavg'].round(1)

In [12]:
df.isna().sum()

tavg    0
dtype: int64

In [13]:
df.to_csv('chenai_cleaned1.csv',index=False)

In [14]:
df['tavg'].describe()

count    11894.000000
mean        28.490600
std          2.606517
min         20.900000
25%         26.300000
50%         28.700000
75%         30.400000
max         36.600000
Name: tavg, dtype: float64

In [15]:
fd=df

# Data Partitioning

In [19]:
train = fd.iloc[:-30]
test = fd.iloc[-30:]

In [21]:
train.tail()

,tavg
time,
2022-06-21,28.4
2022-06-22,28.8
2022-06-23,29.0
2022-06-24,29.1
2022-06-25,29.5


In [17]:
train = fd.iloc[:-30]
test = fd.iloc[-30:]

In [18]:
test

,tavg
time,
2022-06-26,29.9
2022-06-27,29.9
2022-06-28,29.6
2022-06-29,29.5
2022-06-30,30.0
2022-07-01,29.3
2022-07-02,29.4
2022-07-03,28.9
2022-07-04,29.9


# ARIMA MODEL

# ARIMA (6,0,2)

In [22]:
model_arima = ARIMA(train['tavg'],order = (6,0,2))
model_arima = model_arima.fit()
model_arima.summary()

/home/adarsh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/adarsh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/adarsh/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   tavg   No. Observations:                11864
Model:                 ARIMA(6, 0, 2)   Log Likelihood              -15445.814
Date:                Thu, 08 Aug 2024   AIC                          30911.629
Time:                        16:26:22   BIC                          30985.441
Sample:                    01-01-1990   HQIC                         30936.403
                         - 06-25-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.4773      0.384     74.099      0.000      27.724      29.231
ar.L1          0.6072      0.024     25.502      0.000       0.560       0.654
ar.L2          0.9393      0.031     29.995      0.000       0.878       1.001
ar.L3         -0.5803      0.020    -28.869      0.000      -0.620      -0.541
ar.L4          0.0080      0.012      0.641      0.522      -0.016       0.032
ar.L5          0.0197      0.011      1.793      0.073      -0.002       0.041
ar.L6         -0.0006      0.011     -0.055      0.956      -0.021       0.020
ma.L1          0.1081      0.023      4.775      0.000       0.064       0.152
ma.L2         -0.8119      0.022    -36.149      0.000      -0.856      -0.768
sigma2         0.7911      0.008    103.547      0.000       0.776       0.806
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              2805.18
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.65   Skew:                            -0.56
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.10
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [23]:
start = len(train)
end = len(train)+len(test)-1
pred_arima = model_arima.predict(start=start,end=end)
print(pred_arima)

2022-06-26    29.447787
2022-06-27    29.365105
2022-06-28    29.349545
2022-06-29    29.297793
2022-06-30    29.307138
2022-07-01    29.271309
2022-07-02    29.286639
2022-07-03    29.256198
2022-07-04    29.271969
2022-07-05    29.243985
2022-07-06    29.258882
2022-07-07    29.232570
2022-07-08    29.246343
2022-07-09    29.221451
2022-07-10    29.234101
2022-07-11    29.210508
2022-07-12    29.222093
2022-07-13    29.199713
2022-07-14    29.210300
2022-07-15    29.189059
2022-07-16    29.198713
2022-07-17    29.178544
2022-07-18    29.187328
2022-07-19    29.168166
2022-07-20    29.176139
2022-07-21    29.157926
2022-07-22    29.165142
2022-07-23    29.147822
2022-07-24    29.154333
2022-07-25    29.137853
Freq: D, Name: predicted_mean, dtype: float64


In [24]:
import pickle

with open('./chenai_model.pkl', 'wb') as file:
    pickle.dump(model_arima, file)